In [ ]:
library(readr)
library(dplyr)
library(tidyr)
library(stringr)
library(magrittr)
library(padr)
library(kableExtra)
library(purrr)
library(ggplot2)
library(ggthemr)
library(here)
library(lubridate)
# library(caret)
# library(scales)
ggthemr("fresh")
devtools::load_all()

In [ ]:
load(paste0(here(), "/data/other/species.RData"))
load(paste0(here(), "/data/other/stations.RData"))

In [ ]:
data_dwh <- import_data_dwh(paste0(here(), "/data/dwh/pollen_dwh_daily.txt"))
  

In [ ]:


data_dwh <- import_data_dwh(paste0(here(), "/data/dwh/pollen_dwh_daily.txt"))

data_alnu20 <- import_data_cosmo("/scratch/sadamov/wd/20_alnu_pheno_v2/mod_pollen_combined.txt", type = "Phenology V2")
data_alnu20 %<>% mutate(value = 0.6 * value)
data_alnu19 <- import_data_cosmo("/scratch/sadamov/wd/19_alnu_pheno_v2/mod_pollen_combined.txt", type = "Phenology V2")
data_alnu19 %<>% mutate(value = 0.6 * value)
data_betu20 <- import_data_cosmo("/scratch/sadamov/wd/20_betu_pheno_v2/mod_pollen_combined.txt", type = "Phenology V2")
data_betu19 <- import_data_cosmo("/scratch/sadamov/wd/19_betu_pheno_v2/mod_pollen_combined.txt", type = "Phenology V2")
data_poac20 <- import_data_cosmo("/scratch/sadamov/wd/20_poac_pheno_v2/mod_pollen_combined.txt", type = "Phenology V2")
data_poac19 <- import_data_cosmo("/scratch/sadamov/wd/19_poac_pheno_v2/mod_pollen_combined.txt", type = "Phenology V2")

data_list <- list(
  alnu20 = data_alnu20,
  alnu19 = data_alnu19,
  betu20 = data_betu20,
  betu19 = data_betu19,
  poac20 = data_poac20,
  poac19 = data_poac19
)

data_daily_list <- map(data_list, ~ .x %>%
  aggregate_pollen() %>%
  impute_daily() %>%
  filter(measurement == "concentration"))

data_daily <- data_daily_list %>% bind_rows()

data_dwh %<>% impute_daily()

data_daily_comp <- data_dwh %>%
  select(date, taxon, station, obs = value) %>%
  right_join(data_daily, by = c("taxon", "date", "station")) %>%
  select(date, taxon, station, obs, value)

In [ ]:
stations$station
pol_tuning <- tibble(taxon = rep(c("Alnus", "Betula", "Poaceae"), each = 14),
  tuning_orig = rep(c(0.34, 0.63, 0.26), each = 14),
  station = rep(stations$station, times = 3))

data_daily_comp %>%
  group_by(station, taxon) %>%
  summarise(obs = sum(obs, na.rm = TRUE),
            value = sum(value, na.rm = TRUE)) %>%
  ungroup() %>%
  mutate(tuning = obs / value) %>%
  left_join(pol_tuning, by = c("station", "taxon")) %>%
  mutate(tuning_new = tuning * tuning_orig) %>%
  arrange(taxon, station)